In [23]:
"""adjust average prices for inflation using the CPI
stores results in 'IA Average Price' column
adjusts relative to last date - i.e. 2018 dollars throughout
"""

"adjust average prices for inflation using the CPI\nstores results in 'IA Average Price' column\nadjusts relative to last date - i.e. 2018 dollars throughout\n"

In [24]:
import fbprophet as fp
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import RandomizedSearchCV
sns.set_style("darkgrid")
import time
import math
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
import random

In [25]:
# veggie and city data
datadir = './concat_data/'
veggienames = ['APPLES','CARROTS']
regions = ['NORTHEAST+U.S.', 'SOUTHWEST+U.S.']

In [26]:
def load_and_clean(region, veg, dir='./concat_data/'):
    filepath = dir + region + "_" + veg + "_ALL.csv"
    try:
        df = pd.read_csv(filepath, parse_dates=True, index_col='Date')
    except FileNotFoundError:
        print("No data found for {} {}, skipping")
        return None, False
    
    #Drop null rows
    if sum(df.index.isna() == True) > 0:
        df.drop(df[df.index.isna() == True].index, inplace=True)
    # Drop Unnamed column
    if 'Unnamed: 0' in df.columns:
        df.drop(['Unnamed: 0'], axis=1, inplace=True)
    return df, True

In [27]:
def nearest_date(dates, targdate):
    # given a pd series of dates and a target date, returns date from the series closest to target date (and distance)
    for i in dates:
        i = i.to_pydatetime()
    nearest = min(dates, key=lambda x: abs(x - targdate))
    timedelta = abs(nearest - targdate)
    return nearest, timedelta

In [28]:
# read in the cpid inverse coefficients
coeffs = pd.read_csv('./CPI_DATA.csv')
coeffs['DATE'] = pd.to_datetime(coeffs['DATE'])
coeffs = coeffs.sort_values(by='DATE')
coeffs = coeffs.reset_index(drop=True)

In [29]:
# now loop over the data
apples = load_and_clean('NORTHEAST+U.S.', 'APPLES')[0]
apples = apples.reset_index().sort_values(by='Date')
apples.head(20)
        

,Date,Region,Class,Commodity,Variety,Organic,Environment,Unit,Number of Stores,Weighted Avg Price,Low Price,High Price,% Marked Local
0,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,BRAEBURN,NaN,NaN,per pound,125.0,1.24,0.88,1.49,NaN
19,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,RED DELICIOUS,Y,NaN,per pound,502.0,1.70,1.29,1.99,NaN
18,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,RED DELICIOUS,Y,NaN,3 lb bag,168.0,3.33,3.00,3.99,NaN
17,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,RED DELICIOUS,NaN,NaN,per pound,1010.0,1.37,0.89,1.79,NaN
16,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,RED DELICIOUS,NaN,NaN,3 lb bag,254.0,2.99,2.99,2.99,NaN
15,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,PINK LADY/CRIPPS PINK,Y,NaN,per pound,17.0,2.49,2.49,2.49,NaN
13,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,GRANNY SMITH,Y,NaN,3 lb bag,112.0,3.00,3.00,3.00,NaN
12,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,GRANNY SMITH,NaN,NaN,per pound,509.0,1.24,0.98,1.49,NaN
11,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,GOLDEN DELICIOUS,Y,NaN,per pound,210.0,1.99,1.99,1.99,NaN
10,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,GOLDEN DELICIOUS,NaN,NaN,per pound,828.0,1.36,0.88,1.79,NaN


In [30]:
merged_df = pd.merge_asof(apples, coeffs, left_on='Date', right_on='DATE')
merged_df.head(20)

,Date,Region,Class,Commodity,Variety,Organic,Environment,Unit,Number of Stores,Weighted Avg Price,Low Price,High Price,% Marked Local,DATE,CPIAUCNS
0,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,BRAEBURN,NaN,NaN,per pound,125.0,1.24,0.88,1.49,NaN,2011-01-01,220.223
1,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,RED DELICIOUS,Y,NaN,per pound,502.0,1.70,1.29,1.99,NaN,2011-01-01,220.223
2,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,RED DELICIOUS,Y,NaN,3 lb bag,168.0,3.33,3.00,3.99,NaN,2011-01-01,220.223
3,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,RED DELICIOUS,NaN,NaN,per pound,1010.0,1.37,0.89,1.79,NaN,2011-01-01,220.223
4,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,RED DELICIOUS,NaN,NaN,3 lb bag,254.0,2.99,2.99,2.99,NaN,2011-01-01,220.223
5,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,PINK LADY/CRIPPS PINK,Y,NaN,per pound,17.0,2.49,2.49,2.49,NaN,2011-01-01,220.223
6,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,GRANNY SMITH,Y,NaN,3 lb bag,112.0,3.00,3.00,3.00,NaN,2011-01-01,220.223
7,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,GRANNY SMITH,NaN,NaN,per pound,509.0,1.24,0.98,1.49,NaN,2011-01-01,220.223
8,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,GOLDEN DELICIOUS,Y,NaN,per pound,210.0,1.99,1.99,1.99,NaN,2011-01-01,220.223
9,2011-01-07,NORTHEAST U.S.,FRUITS,APPLES,GOLDEN DELICIOUS,NaN,NaN,per pound,828.0,1.36,0.88,1.79,NaN,2011-01-01,220.223


In [31]:
merged_df.iloc[5000:5100]

,Date,Region,Class,Commodity,Variety,Organic,Environment,Unit,Number of Stores,Weighted Avg Price,Low Price,High Price,% Marked Local,DATE,CPIAUCNS
5000,2016-03-11,NORTHEAST U.S.,FRUITS,APPLES,ROME,NaN,NaN,per pound,59.0,1.49,1.49,1.49,NaN,2016-03-01,238.132
5001,2016-03-11,NORTHEAST U.S.,FRUITS,APPLES,GRANNY SMITH,Y,NaN,per pound,139.0,2.88,2.49,2.99,NaN,2016-03-01,238.132
5002,2016-03-11,NORTHEAST U.S.,FRUITS,APPLES,GOLDEN DELICIOUS,NaN,NaN,per pound,250.0,1.50,0.97,1.69,NaN,2016-03-01,238.132
5003,2016-03-11,NORTHEAST U.S.,FRUITS,APPLES,GOLDEN DELICIOUS,NaN,NaN,5 lb bag,167.0,3.99,3.99,3.99,NaN,2016-03-01,238.132
5004,2016-03-11,NORTHEAST U.S.,FRUITS,APPLES,GOLDEN DELICIOUS,NaN,NaN,3 lb bag,188.0,2.96,2.50,3.50,NaN,2016-03-01,238.132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,2016-04-15,NORTHEAST U.S.,FRUITS,APPLES,PINK LADY/CRIPPS PINK,NaN,NaN,per pound,284.0,1.50,0.99,2.29,NaN,2016-04-01,239.261
5096,2016-04-15,NORTHEAST U.S.,FRUITS,APPLES,RED DELICIOUS,NaN,NaN,3 lb bag,197.0,2.67,1.99,2.99,NaN,2016-04-01,239.261
5097,2016-04-15,NORTHEAST U.S.,FRUITS,APPLES,RED DELICIOUS,Y,NaN,per pound,6.0,2.49,2.49,2.49,NaN,2016-04-01,239.261
5098,2016-04-15,NORTHEAST U.S.,FRUITS,APPLES,RED DELICIOUS,NaN,NaN,per pound,674.0,1.41,0.88,1.79,NaN,2016-04-01,239.261


In [32]:
merged_df["IA Average Price"] = (merged_df['Weighted Avg Price']/merged_df['CPIAUCNS'])*100
merged_df = merged_df.set_index('Date')
print('Data with inflation index added:\n', merged_df.head())

Data with inflation index added:
                     Region   Class Commodity        Variety Organic  \
Date                                                                  
2011-01-07  NORTHEAST U.S.  FRUITS    APPLES       BRAEBURN     NaN   
2011-01-07  NORTHEAST U.S.  FRUITS    APPLES  RED DELICIOUS       Y   
2011-01-07  NORTHEAST U.S.  FRUITS    APPLES  RED DELICIOUS       Y   
2011-01-07  NORTHEAST U.S.  FRUITS    APPLES  RED DELICIOUS     NaN   
2011-01-07  NORTHEAST U.S.  FRUITS    APPLES  RED DELICIOUS     NaN   

            Environment       Unit  Number of Stores  Weighted Avg Price  \
Date                                                                       
2011-01-07          NaN  per pound             125.0                1.24   
2011-01-07          NaN  per pound             502.0                1.70   
2011-01-07          NaN   3 lb bag             168.0                3.33   
2011-01-07          NaN  per pound            1010.0                1.37   
2011-01-07  

In [33]:
merged_df.tail()

,Region,Class,Commodity,Variety,Organic,Environment,Unit,Number of Stores,Weighted Avg Price,Low Price,High Price,% Marked Local,DATE,CPIAUCNS,IA Average Price
Date,,,,,,,,,,,,,,,
2021-02-19,NORTHEAST U.S.,FRUITS,APPLES,GRANNY SMITH,Y,NaN,2 lb bag,260.0,3.99,3.99,3.99,NaN,2021-01-01,261.582,1.525334
2021-02-19,NORTHEAST U.S.,FRUITS,APPLES,GRANNY SMITH,Y,NaN,per pound,75.0,1.99,1.99,1.99,NaN,2021-01-01,261.582,0.760756
2021-02-19,NORTHEAST U.S.,FRUITS,APPLES,HONEYCRISP,NaN,NaN,3 lb bag,9.0,2.99,2.99,2.99,NaN,2021-01-01,261.582,1.143045
2021-02-19,NORTHEAST U.S.,FRUITS,APPLES,GALA,NaN,NaN,3 lb bag,58.0,3.27,2.99,4.99,NaN,2021-01-01,261.582,1.250086
2021-02-19,NORTHEAST U.S.,FRUITS,APPLES,RED DELICIOUS,NaN,NaN,per pound,54.0,1.39,1.29,1.49,NaN,2021-01-01,261.582,0.531382
